<a href="https://colab.research.google.com/github/PDGamerSG/GDSC-Image_Classification/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -q kaggle

In [15]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"pdgamersg","key":"35378d4ad319c63c6a143638f767f63c"}'}

In [16]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [17]:
!cp kaggle.json ~/.kaggle/

In [18]:
! chmod 600 ~/.kaggle/kaggle.json

In [19]:
!kaggle datasets list

ref                                                              title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
abdulmalik1518/mobiles-dataset-2025                              Mobiles Dataset (2025)                            20KB  2025-02-18 06:50:24           6011        103  1.0              
asinow/car-price-dataset                                         Car Price Dataset                                135KB  2025-01-26 19:53:28          22792        365  1.0              
mahmoudelhemaly/students-grading-dataset                         Student Performance & Behavior Dataset           508KB  2025-02-17 17:38:46           4853         93  1.0              
samikshadalvi/lungs-diseases-dataset                             Lungs

In [20]:
!kaggle competitions download -c cifar-10

cifar-10.zip: Skipping, found more recently modified local copy (use --force to force download)


In [21]:
!unzip cifar-10.zip

Archive:  cifar-10.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [23]:
!apt-get install p7zip-full
!7z x train.7z

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 109723070 bytes (105 MiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 109723070
Headers Size = 294768
Method = LZMA:26
Solid = +
Blocks = 1

  0%      2%      2% 782 - train/10701.png                            2% 1277 - train/11147.png                             2% 1662 - train/11494.png                           

In [24]:
!apt-get install p7zip-full
!7z x test.7z

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 639374249 bytes (610 MiB)

Extracting archive: test.7z
--
Path = test.7z
Type = 7z
Physical Size = 639374249
Headers Size = 1721070
Method = LZMA:26
Solid = +
Blocks = 1

  0%      0% - test/1.png                   0% 976 - test/100876.png                            0% 1429 - test/101283.png                             0% 1764 - test/101585.png

In [ ]:
import os
import shutil
import pandas as pd
df = pd.read_csv('trainLabels.csv')
source_folder = 'train'

categories = df['label'].unique()
for category in categories:
    os.makedirs(category, exist_ok=True)

for index, row in df.iterrows():
    filename = f"{row['id']}.png" #f does that same thing only give value and then add at last png to make it a image
    label = row['label']

    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(label, filename)

    if os.path.exists(source_file):
        shutil.move(source_file, destination_file) #helps to move the file from the location
        print(f"Moved {filename} to {label} folder")
    else:
        print(f"Warning: File {filename} not found")

print("Image sorting complete")


In [39]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.classes = sorted(os.listdir(folder_path))
        self.image_paths = []

        for class_index, class_name in enumerate(self.classes):
            class_folder = os.path.join(folder_path, class_name)
            for img_file in os.listdir(class_folder):
                self.image_paths.append((os.path.join(class_folder, img_file), class_index))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_path, label = self.image_paths[index]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = ImageDataset(folder_path='/content/All Categories', transform=transform)

data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

print(f"Total images: {len(dataset)}")
print(f"Classes: {dataset.classes}")
print(f"Number of batches: {len(data_loader)}")

for class_name in dataset.classes:
    class_folder = os.path.join('/content/All Categories', class_name)
    num_images = len(os.listdir(class_folder))
    print(f"{class_name}: {num_images} images")

for images, labels in data_loader:
    print(f"Batch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    break


Total images: 50000
Classes: ['.ipynb_checkpoints', 'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'train', 'truck']
Number of batches: 782
.ipynb_checkpoints: 0 images
airplane: 5000 images
automobile: 5000 images
bird: 5000 images
cat: 5000 images
deer: 5000 images
dog: 5000 images
frog: 5000 images
horse: 5000 images
ship: 5000 images
train: 0 images
truck: 5000 images
Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
